Referring from Analytics Vidya page

In [1]:
import nltk
import pandas as pd
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from keras import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split

In [12]:
df = pd.read_json("/content/Cell_Phones_and_Accessories_5.json", lines= True)

Preprocessing the data

In [19]:
def data_cleaning( data, stop_words ):
  data['reviewText'] = data['reviewText'].apply( lambda x : ' '.join( x.lower() for x in x.split() ))
  #Removing stop words
  data['reviewText'] = data['reviewText'].apply( lambda x : ' '.join( x for x in x.split() if x not in stop_words ))
  #Removing digits
  data['reviewText'] = data['reviewText'].str.replace('d', '')
  #Lemmatization
  #data['reviewText'] = data['reviewText'].apply( lambda x : ' '.join( Word(x).lemmatize for x in x.split() ))

In [4]:
from nltk.corpus import stopwords

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [28]:
stop_words = stopwords.words('english')
data_cleaning( df, stop_words )

In [27]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",look goo stick goo! like roune shape always bu...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",stickers work like review says o. stick great ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",awesome make phone look stylish! use one far a...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",item arrive great time perfect conition. howev...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, looks great. use multiple a...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [29]:
df['reviewText']

0         look goo stick goo! like roune shape always bu...
1         stickers work like review says o. stick great ...
2         awesome make phone look stylish! use one far a...
3         item arrive great time perfect conition. howev...
4         awesome! stays on, looks great. use multiple a...
                                ...                        
194434    works great like original one. really nee extr...
194435    great prouct. great packaging. high quality ap...
194436    great cable, goo expensive apple one. husban a...
194437    really like becasue works well life proof case...
194438    prouct escribe, waste lot money cors. woul buy...
Name: reviewText, Length: 194439, dtype: object

In [30]:
tokenizer = Tokenizer( num_words=500, split=" ")
tokenizer.fit_on_texts( df['reviewText'].values )
X = tokenizer.texts_to_sequences( df['reviewText'].values )
X = pad_sequences(X)

In [37]:
model = Sequential()
model.add( Embedding( 500, 120, input_length= X.shape[1] ) )
model.add( SpatialDropout1D(0.4) )
model.add( LSTM( 704 , dropout=0.2, recurrent_dropout= 0.2) )
model.add( Dense( 352, activation= 'LeakyReLU') )
model.add( Dense( 1, activation= 'softmax') )
model.compile( loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'] )
print( model.summary() )

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1461, 120)         60000     
                                                                 
 spatial_dropout1d_2 (Spati  (None, 1461, 120)         0         
 alDropout1D)                                                    
                                                                 
 lstm_2 (LSTM)               (None, 704)               2323200   
                                                                 
 dense_4 (Dense)             (None, 352)               248160    
                                                                 
 dense_5 (Dense)             (None, 1)                 353       
                                                                 
Total params: 2631713 (10.04 MB)
Trainable params: 2631713 (10.04 MB)
Non-trainable params: 0 (0.00 Byte)
______________

In [33]:
X_train, X_test, y_train, y_test = train_test_split( X, df['overall'], test_size=0.25)

In [ ]:
#Training
model.fit(X_train, y_train)

#Evaluate
model.evaluate(X_test)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)
